In [3]:
import sys
sys.path.append('/Users/wugaomin/MyDocument/NYUSpring2021/Tools_ML/Project/slates_semisynth_expts/')

In [4]:
import numpy
import GammaDP
import scipy.linalg

A toy example of two slots with three documents for ranking

In [5]:
m = 3
l = 2
num_candidates = m
validDocs = l


In [6]:
# decay is equal to 0.0 as uniform target policy
decay = 0.0
multinomial=numpy.arange(1, num_candidates+1, dtype=numpy.float64)
multinomial=numpy.exp2((-decay)*numpy.floor(numpy.log2(multinomial)))

gamma=None
gammaVals=GammaDP.GammaCalculator(multinomial.tolist(), validDocs)
gamma=numpy.diag(numpy.ravel(gammaVals.unitMarginals))

In [7]:
for p in range(validDocs):
    for q in range(p+1, validDocs):
        pairMarginals=gammaVals.pairwiseMarginals[(p,q)]
        currentRowStart=p*num_candidates
        currentRowEnd=(p+1)*num_candidates
        currentColumnStart=q*num_candidates
        currentColumnEnd=(q+1)*num_candidates
        gamma[currentRowStart:currentRowEnd, currentColumnStart:currentColumnEnd]=pairMarginals
        gamma[currentColumnStart:currentColumnEnd, currentRowStart:currentRowEnd]=pairMarginals.T

In [8]:
normalizer=numpy.sum(multinomial, dtype=numpy.longdouble)
multinomial=multinomial/normalizer
print("For a uniform policy, choose from 3 docs:", multinomial)

For a uniform policy, choose from 3 docs: [0.33333333 0.33333333 0.33333333]


gamma is $\mathbb{E}_{\mu}\left[\mathbf{1}_{\mathbf{s}} \mathbf{1}_{\mathbf{s}}^{T} \mid x\right]$ in the paper.

In [19]:
print("Unifrom policy gamma implemented from the paper")
print(gamma)

Unifrom policy gamma implemented from the paper
[[0.33333333 0.         0.         0.         0.16666667 0.16666667]
 [0.         0.33333333 0.         0.16666667 0.         0.16666667]
 [0.         0.         0.33333333 0.16666667 0.16666667 0.        ]
 [0.         0.16666667 0.16666667 0.33333333 0.         0.        ]
 [0.16666667 0.         0.16666667 0.         0.33333333 0.        ]
 [0.16666667 0.16666667 0.         0.         0.         0.33333333]]


In [29]:
# doc = 0, 1, 2
tag = [(1,0),(0,1), (1,2),(2,1), (2,0),(0,2)]
# all possible 6 slates 
l = [[0, 1, 0, 1, 0, 0], # (1,0)
     [1, 0, 0, 0, 1, 0], # (0,1)
     [0, 1, 0, 0, 0, 1], # (1,2)
     [0, 0, 1, 0, 1, 0], # (2,1)
     [0, 0, 1, 1, 0, 0], # (2,0)
     [1, 0, 0, 0, 0, 1] # (0,2)
    ]

In [18]:
import numpy as np
mat = 0
for x in l:
    x = np.array(x)
    mat += np.dot(x.reshape(-1, 1), x.reshape(1, -1))
print("Unifrom policy gamma implemented from the equation")
print((1/6.)*mat)

Unifrom policy gamma implemented from the equation
[[0.33333333 0.         0.         0.         0.16666667 0.16666667]
 [0.         0.33333333 0.         0.16666667 0.         0.16666667]
 [0.         0.         0.33333333 0.16666667 0.16666667 0.        ]
 [0.         0.16666667 0.16666667 0.33333333 0.         0.        ]
 [0.16666667 0.         0.16666667 0.         0.33333333 0.        ]
 [0.16666667 0.16666667 0.         0.         0.         0.33333333]]


How to interpret this: says if the 1_s start with 1, meaning the first slot choose a, then the frist slot choose b and c is 0.0 and 0.0.
And the probability that the second slot choose a is also 0.0. 

In [21]:
gammaInv=scipy.linalg.pinv(gamma)
print(gammaInv)

[[-3.6535891e+15 -3.6535891e+15 -3.6535891e+15  3.6535891e+15
   3.6535891e+15  3.6535891e+15]
 [-3.6535891e+15 -3.6535891e+15 -3.6535891e+15  3.6535891e+15
   3.6535891e+15  3.6535891e+15]
 [-3.6535891e+15 -3.6535891e+15 -3.6535891e+15  3.6535891e+15
   3.6535891e+15  3.6535891e+15]
 [ 3.6535891e+15  3.6535891e+15  3.6535891e+15 -3.6535891e+15
  -3.6535891e+15 -3.6535891e+15]
 [ 3.6535891e+15  3.6535891e+15  3.6535891e+15 -3.6535891e+15
  -3.6535891e+15 -3.6535891e+15]
 [ 3.6535891e+15  3.6535891e+15  3.6535891e+15 -3.6535891e+15
  -3.6535891e+15 -3.6535891e+15]]


The inverse of a non-negative matrix is usually **not non-negative** (means it's usually have negative entries.). The exception is the non-negative monomial matrices: a non-negative matrix has non-negative inverse if and only if it is a (non-negative) monomial matrix. Note that thus the inverse of a positive matrix is not positive or even non-negative

In [31]:
# denominator is fixed
posRelVector = numpy.array([0, 1, 0, 1, 0, 0])
d = {}
# what would happen if we change the nominator
for idx, newSlateVector in enumerate(l):
    newSlateVector = numpy.array(newSlateVector)
    estimatedPhi=numpy.dot(gammaInv, posRelVector)
#     new_cov = newSlateVector.reshape(-1,1) * (newSlateVector.reshape(-1,1).T)
#     print(new_cov)
#     print(gammaInv*posRelVector*newSlateVector)
    
    invPropensity=numpy.dot(estimatedPhi, newSlateVector)
    key = tag[idx]
    d[key] = invPropensity
    print(newSlateVector)
    print(invPropensity)

[0 1 0 1 0 0]
4.5
[1 0 0 0 1 0]
0.5
[0 1 0 0 0 1]
1.0
[0 0 1 0 1 0]
0.0
[0 0 1 1 0 0]
1.5
[1 0 0 0 0 1]
-1.5


So make sense here, the inv Propensity can be zero.

In [34]:
import pandas as pd
pd.Series(d)

1  0    4.5
0  1    0.5
1  2    1.0
2  1    0.0
   0    1.5
0  2   -1.5
dtype: float64

What if we do IPS here

In [ ]:
l = [[0, 1, 0, 1, 0, 0], # (1,0)
     [1, 0, 0, 0, 1, 0], # (0,1)
     [0, 1, 0, 0, 0, 1], # (1,2)
     [0, 0, 1, 0, 1, 0], # (2,1)
     [0, 0, 1, 1, 0, 0], # (2,0)
     [1, 0, 0, 0, 0, 1] # (0,2)
    ]


In [5]:
m = 3
l = 2
num_candidates = m
validDocs = l

In [26]:
# denominator is fixed
posRelVector = numpy.array([0, 1, 0, 1, 0, 0])

# what would happen if we change the nominator
for newSlateVector in l:
    exactMatch=numpy.absolute(newSlateVector-posRelVector).sum() == 0
    invPropensity=0.0
    if exactMatch:
        invPropensity=numpy.prod(range(num_candidates+1-validDocs, num_candidates+1), dtype=numpy.float64)
        
    print(newSlateVector)
    print(invPropensity)

[0, 1, 0, 1, 0, 0]
6.0
[1, 0, 0, 0, 1, 0]
0.0
[0, 1, 0, 0, 0, 1]
0.0
[0, 0, 1, 0, 1, 0]
0.0
[0, 0, 1, 1, 0, 0]
0.0
[1, 0, 0, 0, 0, 1]
0.0
